# Row vs Column Storage

In [27]:
import iris
import time
import csv
import random
import datetime
import string
import numpy as np

## Helper Functions

In [28]:
# benchmark an sql query
def benchmark_sql_query(sql_query):
    start = time.time()
    iris.sql.exec(sql_query)
    end = time.time()
    print(f"{sql_query} in {end - start}")

# print result of sql query
def print_sql_query(sql_query):
    print(f"{sql_query} :")
    rs = iris.sql.exec(sql_query)
    for row in rs:
        print(row)


# create n fake data
def create_n_fake_data(n):


    def random_string(string_length=10):
        letters = string.ascii_lowercase
        return ''.join(random.choice(letters) for i in range(string_length))

    def random_date(start, end):
        return start + datetime.timedelta(
            # Get a random amount of seconds between `start` and `end`
            seconds=random.randint(0, int((end - start).total_seconds())),
        )

    def random_amount():
        return np.random.uniform(0, 10000)

    def random_type():
        return random.choice(['credit', 'debit'])

    def random_account_number():
        return random.randint(1000000, 9999999)

    data = []
    # header
    data.append(['AccountNumber', 'TransactionDate', 'Description', 'Amount', 'Type'])
    for i in range(n):
        data.append([
            random_account_number(),
            random_date(datetime.datetime(2018, 1, 1), datetime.datetime(2019, 1, 1)),
            random_string(10),
            random_amount(),
            random_type()
        ])
    return data


## List of tables

In [29]:
list_tables = ["Demo.BankTransactionRow", "Demo.BankTransactionColumn","Demo.BankTransactionMix" ]

In [30]:
# init drop table if exists
print("init drop table if exists")
for table in list_tables:
    benchmark_sql_query("DROP TABLE IF EXISTS %s" % table)

init drop table if exists
DROP TABLE IF EXISTS Demo.BankTransactionRow in 0.09867167472839355
DROP TABLE IF EXISTS Demo.BankTransactionColumn in 0.04582977294921875
DROP TABLE IF EXISTS Demo.BankTransactionMix in 0.04502701759338379


In [31]:
# create tables
# row storage
sql_row = """
CREATE TABLE Demo.BankTransactionRow (
  AccountNumber INTEGER,
  TransactionDate DATE,
  Description VARCHAR(100),
  Amount NUMERIC(10,2),
  Type VARCHAR(10)
)
"""

In [32]:
# column storage
sql_column = """
CREATE TABLE Demo.BankTransactionColumn (
  AccountNumber INTEGER,
  TransactionDate DATE,
  Description VARCHAR(100),
  Amount NUMERIC(10,2),
  Type VARCHAR(10)
)
WITH STORAGETYPE = COLUMNAR
"""

In [33]:
# mix storage
sql_mix = """
CREATE TABLE Demo.BankTransactionMix (
  AccountNumber INTEGER,
  TransactionDate DATE,
  Description VARCHAR(100),
  Amount NUMERIC(10,2) WITH STORAGETYPE = COLUMNAR,
  Type VARCHAR(10)
)
"""

In [34]:
print("create tables")
sqls = [sql_row, sql_column, sql_mix]
for sql in sqls:
    benchmark_sql_query(sql)

create tables

CREATE TABLE Demo.BankTransactionRow (
  AccountNumber INTEGER,
  TransactionDate DATE,
  Description VARCHAR(100),
  Amount NUMERIC(10,2),
  Type VARCHAR(10)
)
 in 0.20645761489868164

CREATE TABLE Demo.BankTransactionColumn (
  AccountNumber INTEGER,
  TransactionDate DATE,
  Description VARCHAR(100),
  Amount NUMERIC(10,2),
  Type VARCHAR(10)
)
WITH STORAGETYPE = COLUMNAR
 in 0.20578598976135254

CREATE TABLE Demo.BankTransactionMix (
  AccountNumber INTEGER,
  TransactionDate DATE,
  Description VARCHAR(100),
  Amount NUMERIC(10,2) WITH STORAGETYPE = COLUMNAR,
  Type VARCHAR(10)
)
 in 0.18250083923339844


In [35]:
print("create data")
number_fake_data = 500000
data = create_n_fake_data(number_fake_data)
with open('data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(data)

print("load data")
for table in list_tables:
    benchmark_sql_query(f"LOAD BULK %NOJOURN DATA FROM FILE '/opt/irisapp/data/data.csv' INTO {table}")

# tune table
print("tune table")
for table in list_tables:
    benchmark_sql_query("TUNE TABLE %s" % table)

create data
load data
LOAD BULK %NOJOURN DATA FROM FILE '/opt/irisapp/data/data.csv' INTO Demo.BankTransactionRow in 3.047476291656494
LOAD BULK %NOJOURN DATA FROM FILE '/opt/irisapp/data/data.csv' INTO Demo.BankTransactionColumn in 44.84366583824158
LOAD BULK %NOJOURN DATA FROM FILE '/opt/irisapp/data/data.csv' INTO Demo.BankTransactionMix in 34.497655153274536
tune table
TUNE TABLE Demo.BankTransactionRow in 0.5842890739440918
TUNE TABLE Demo.BankTransactionColumn in 0.7010197639465332
TUNE TABLE Demo.BankTransactionMix in 0.5431900024414062


In [89]:
for table in list_tables:
    benchmark_sql_query(f"CREATE BITMAP INDEX TypeIndex ON {table}(AccountNumber)")


CREATE BITMAP INDEX TypeIndex ON Demo.BankTransactionRow(AccountNumber) in 2.4024267196655273
CREATE BITMAP INDEX TypeIndex ON Demo.BankTransactionColumn(AccountNumber) in 7.740201473236084
CREATE BITMAP INDEX TypeIndex ON Demo.BankTransactionMix(AccountNumber) in 1.6915180683135986


SQLError: Class 'Demo.BankTransactionIndex' already has index named 'AmountIndex'.

In [111]:
# utility vars and function to compare metrics
last_time = 0
last_grefs = 0
plans = []

def reset_compare():
    global last_time, last_grefs
    last_time = 0
    last_grefs = 0

def brag(prev, new, speed=True):
    if ((prev == 0) or (new == 0) or (prev == new)):
        return ""
    if (prev > new):
        rate = prev / new
        if (rate > 2):
            return " - {:,.2f}x {}".format(rate, "faster" if speed else "less")
        else:
            return " - {:,.0%} {}".format(rate-1, "faster" if speed else "less")
    else:
        rate = new / prev
        if (rate > 2):
            return " - {:,.2f}x {}".format(rate, "slower" if speed else "more")
        else:
            return " - {:,.0%} {}".format(rate-1, "slower" if speed else "more")

def just_sql(sql):
    return iris.sql.exec(sql)[0][0]

def query(sql, compare = False, dtypes = None):
    global last_time, last_grefs, plans
    
    start_grefs = just_sql(f"SELECT Demo.GloRefs()")
    
    start_time = time.time()
    result = iris.sql.exec(sql).dataframe()
    exec_time = time.time() - start_time
    
    exec_grefs = just_sql("SELECT Demo.GloRefs()") - start_grefs - 1
    
    if compare:
        print("\nExecution time: "+str(round(exec_time,3))+"s " + brag(last_time, exec_time))
        print("Global references: "+str(exec_grefs)+" "+ brag(last_grefs, exec_grefs, False))
        last_time = exec_time
        last_grefs = exec_grefs
    else:
        print("\nExecution time: "+str(round(exec_time,3))+"s \nGlobal references: "+str(exec_grefs))
    
    return result

In [102]:
print("tune table")
for table in list_tables:
    benchmark_sql_query(f"BUILD INDEX FOR TABLE {table}")
    benchmark_sql_query("TUNE TABLE %s" % table)

tune table
BUILD INDEX FOR TABLE Demo.BankTransactionRow in 1.4768145084381104
TUNE TABLE Demo.BankTransactionRow in 0.618048906326294
BUILD INDEX FOR TABLE Demo.BankTransactionColumn in 7.12926983833313
TUNE TABLE Demo.BankTransactionColumn in 0.6982026100158691
BUILD INDEX FOR TABLE Demo.BankTransactionMix in 1.1216766834259033
TUNE TABLE Demo.BankTransactionMix in 0.5242881774902344


In [112]:
# query data
print("query data")
iris.sql.exec('PURGE CACHED QUERIES')
for table in list_tables:
    query(f"SELECT Amount FROM {table}")


query data


SQLError:  User defined SQL function 'DEMO.GLOREFS' does not exist

In [104]:
# benchmark aggregation
print("benchmark aggregation")
iris.sql.exec('PURGE CACHED QUERIES')
for table in list_tables:
    benchmark_sql_query(f"SELECT AVG(ABS(Amount)),AVG(AccountNumber) FROM {table} where Type = 'credit' and TransactionDate < now()")


benchmark aggregation
SELECT AVG(ABS(Amount)),AVG(AccountNumber) FROM Demo.BankTransactionRow where Type = 'credit' and TransactionDate < now() in 0.1443159580230713
SELECT AVG(ABS(Amount)),AVG(AccountNumber) FROM Demo.BankTransactionColumn where Type = 'credit' and TransactionDate < now() in 0.10837221145629883
SELECT AVG(ABS(Amount)),AVG(AccountNumber) FROM Demo.BankTransactionMix where Type = 'credit' and TransactionDate < now() in 0.10695695877075195


In [49]:
# table size
print("table size")
for table in list_tables:
    print(table)
    print_sql_query(f"SELECT * FROM bdb_sql.TableSize('{table}')") 
table = 'Demo.BankTransactionIndex'

table size
Demo.BankTransactionRow
SELECT * FROM bdb_sql.TableSize('Demo.BankTransactionRow') :
Total size: 21.055 allocated, 19.046 used
['^CATa.BArF.1', '21', '19']
['^CATa.BArF.2', '0.055', '0.046']
['^CATa.BArF.S', '', '']
['total', '21.055', '19.046']
Demo.BankTransactionColumn
SELECT * FROM bdb_sql.TableSize('Demo.BankTransactionColumn') :
['^CATa.BJqo.1', '4.3', '3.8']
['^CATa.BJqo.1.V1', '2.0', '2.0']
['^CATa.BJqo.1.V2', '2.0', '2.0']
['^CATa.BJqo.1.V3', '18', '18']
['^CATa.BJqo.1.V4', '2.0', '2.0']
['^CATa.BJqo.1.V5', '0.56', '0.54']
['^CATa.BJqo.2', '0.055', '0.046']
['^CATa.BJqo.S', '', '']
['total', '28.915', '28.386']
Demo.BankTransactionMix
SELECT * FROM bdb_sql.TableSize('Demo.BankTransactionMix') :
['^CATa.CfQt.1', '19', '17']
['^CATa.CfQt.1.V1', '2.0', '2.0']
['^CATa.CfQt.2', '0.055', '0.046']
['^CATa.CfQt.S', '', '']
['total', '21.055', '19.046']
Demo.BankTransactionIndex
Total size: 28.915 allocated, 28.386 used


1

Total size: 21.055 allocated, 19.046 used
Total size: 34.055 allocated, 28.646 used
